In [1]:
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mpldts
import calendar
import csv

In [2]:
stn = '154' #067
dataset = 'archive' # Enter 'archive' or 'realtime'
deploy = '1' # If archive dataset, set deployment number from .nc file

start_date = '10/09/2017 9:00' # MM/DD/YYYY HH:MM
duration  = 30 # Set length of timeseries (minutes)

qc_level = 2 # Filter data with qc flags above this number

In [3]:
datapath = '../Python/data/154p1_d08.nc'
nc = netCDF4.Dataset(datapath)
# Turn off auto masking
nc.set_auto_mask(False)
nc

<class 'netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    naming_authority: edu.ucsd.cdip
    keywords_vocabulary: Global Change Master Directory (GCMD) Earth Science Keywords
    date_created: 2020-07-29T19:27:25Z
    date_issued: 2020-07-29T19:27:25Z
    date_modified: 2020-07-29T19:27:25Z
    creator_name: Coastal Data Information Program, SIO/UCSD
    creator_url: http://cdip.ucsd.edu
    creator_email: www@cdip.ucsd.edu
    publisher_name: Coastal Data Information Program, SIO/UCSD
    publisher_url: http://cdip.ucsd.edu
    publisher_email: www@cdip.ucsd.edu
    institution: Scripps Institution of Oceanography, University of California San Diego
    project: Coastal Data Information Program (CDIP)
    processing_level: QA/QC information available at http://cdip.ucsd.edu/documentation
    standard_name_vocabulary: CF Standard Name Table (v29, 08 July 2015)
    Conventions: ACDD-1.3, CF-1.6
    license: These data may be redistributed and used with

In [4]:
attribute_value = nc.getncattr('time_coverage_start')
print(attribute_value)

# MM/DD/YYYY HH:MM

type(attribute_value)
MM = attribute_value[5:7]
DD = attribute_value[8:10]
YYYY = attribute_value[:4]
HH = attribute_value[11:13]
mm = attribute_value[14:16]

start_date_1 = MM + '/' + DD + '/' + YYYY + ' ' + HH + ':' + mm
start_date_1

2017-08-02T14:00:00Z


'08/02/2017 14:00'

In [35]:
xdisp = nc.variables['xyzXDisplacement'] # Make a numpy array of three directional displacement variables (x, y, z)
ydisp = nc.variables['xyzYDisplacement']
zdisp = nc.variables['xyzZDisplacement']
qc_flag = nc.variables['xyzFlagPrimary']
filter_delay = nc.variables['xyzFilterDelay']
start_time = nc.variables['xyzStartTime'][:] # Variable that gives start time for buoy data collection
sample_rate = nc.variables['xyzSampleRate'][:] # Variable that gives rate (frequency, Hz) of sampling
end_time = start_time + (len(xdisp)/sample_rate) # Calulate end time for buoy data collection

# Get station name and number for plot title
station_name = nc.variables['metaStationName'][:]
station_title = station_name.tobytes().decode().split('\x00',1)[0]

xdisp

<class 'netCDF4.Variable'>
float32 xyzXDisplacement(xyzCount)
    long_name: sea surface north displacement
    units: meter
    _FillValue: -999.99
    coordinates: metaDeployLatitude metaDeployLongitude
    grid_mapping: metaGridMapping
    valid_min: 20.47
    valid_max: -20.47
    ancillary_variables: xyzFlagPrimary xyzFlagSecondary
    name: x displacement
    comment: UTC measurement times calculated as xyzStartTime + arrayIndex/xyzSampleRate - xyzFilterDelay, where arrayIndex is the displacement array index starting from 0.
unlimited dimensions: 
current shape = (1640447,)
filling on

In [9]:
# Convert to unix timestamp
def get_unix_timestamp(human_time,dateFormat):
    unix_timestamp = int(calendar.timegm(datetime.datetime.strptime(human_time, dateFormat).timetuple()))
    return unix_timestamp

# Convert to human readable timestamp
def get_human_timestamp(unix_timestamp, dateFormat):
    human_timestamp = datetime.datetime.utcfromtimestamp(int(unix_timestamp)).strftime(dateFormat)
    return human_timestamp

data_start = get_human_timestamp(start_time - filter_delay[0],"%m/%d/%Y %H:%M:%S")
data_end = get_human_timestamp(end_time - filter_delay[0],"%m/%d/%Y %H:%M:%S")
print("data_start: " + data_start)
print("  data_end: " + data_end)

data_start: 08/02/2017 13:57:46
  data_end: 08/17/2017 09:57:45


In [13]:
# Find UNIX timestamps for user human-formatted start/end dates
unix_start = get_unix_timestamp(start_date,"%m/%d/%Y %H:%M") 
unix_end = unix_start + (duration * 60) # Create UNIX end stamp by adding duration to 'unix_start'

# Create specialized array using UNIX Start and End times minus Filter Delay, and Sampling Period (1/sample_rate) 
# to calculate sub-second time values that correspond to Z-Displacement sampling values
sample_time = np.arange((start_time - filter_delay[0]), end_time - filter_delay[0],(1/(sample_rate)))

# Find corresponding start/end date index numbers in 'sample_time' array    
start_index = sample_time.searchsorted(unix_start) 
end_index = sample_time.searchsorted(unix_end)

unix_start

1507539600

In [15]:
unix_end

1507541400

In [17]:
start_index

np.int64(1640448)

In [19]:
end_index

np.int64(1640448)

In [21]:
sample_time

array([1.50168227e+09, 1.50168227e+09, 1.50168227e+09, ...,
       1.50296386e+09, 1.50296387e+09, 1.50296387e+09], shape=(1640448,))

In [23]:
xdisp[start_index:end_index]

array([], dtype=float32)

In [26]:
datapath_hist = '../Python/data/154p1_historic.nc'
nc_hist = netCDF4.Dataset(datapath_hist)
# Turn off auto masking
nc_hist.set_auto_mask(False)
nc_hist

<class 'netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    naming_authority: edu.ucsd.cdip
    keywords_vocabulary: Global Change Master Directory (GCMD) Earth Science Keywords
    date_created: 2024-10-28T22:26:43Z
    date_issued: 2024-10-28T22:26:43Z
    date_modified: 2024-10-28T22:26:43Z
    creator_name: Coastal Data Information Program, SIO/UCSD
    creator_url: http://cdip.ucsd.edu
    creator_email: www@cdip.ucsd.edu
    creator_institution: Scripps Institution of Oceanography, UCSD
    creator_country: USA
    creator_sector: academic
    publisher_name: Coastal Data Information Program, SIO/UCSD
    publisher_url: http://cdip.ucsd.edu
    publisher_email: www@cdip.ucsd.edu
    publisher_country: USA
    publisher_institution: Scripps Institution of Oceanography, UCSD
    institution: Scripps Institution of Oceanography, University of California San Diego
    project: Coastal Data Information Program (CDIP)
    processing_level: QA/QC informat

In [28]:
attribute_value_hist = nc_hist.getncattr('time_coverage_start')
print(attribute_value_hist)
#attribute_end_hist = nc.hist.getncattr('time_coverage_end')
#print(attribute_end_hist)

2009-09-17T21:00:00Z


In [33]:
waveTime_hist = nc_hist.variables['waveTime']
print(waveTime_hist)

<class 'netCDF4.Variable'>
int32 waveTime(waveTime)
    long_name: UTC sample start time
    standard_name: time
    units: seconds since 1970-01-01 00:00:00 UTC
    calendar: standard
    axis: T
    coverage_content_type: coordinate
    bounds: waveTimeBounds
    comment: Spectral processing performed on 1600-second samples specified by waveTimeBounds.
unlimited dimensions: 
current shape = (246568,)
filling on, default _FillValue of -2147483647 used


In [38]:
xdisp_hist = nc_hist.variables['xyzXDisplacement']

KeyError: 'xyzXDisplacement'